In [70]:
# import os, getpass
import chromadb
from ibm_watsonx_ai import Credentials, APIClient
from langchain_chroma import Chroma
from langchain_core.documents import Document
from uuid import uuid4
from langchain_text_splitters import MarkdownTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_ibm import WatsonxEmbeddings
from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import DecodingMethods
from langchain_ibm import ChatWatsonx, WatsonxLLM
# from langchain.chains import RetrievalQAjin
from langchain_ibm import WatsonxEmbeddings
from ibm_watsonx_ai.foundation_models.schema import TextChatParameters
from langchain_experimental.text_splitter import SemanticChunker
from langchain_huggingface import HuggingFaceEmbeddings
from transformers import AutoTokenizer
from docling.document_converter import DocumentConverter
from utils import *
PAGE_BREAK = "<!-- page break -->"

In [2]:
api_key="VPryyL4t508YQ_pJQNCU0m23Yctctahrqo2pjT5BMQkV"
url="https://us-south.ml.cloud.ibm.com"
project_id = "6552c3bb-a35a-4bf3-8430-c42828d59a06"

## Load documents

In [ ]:
file_path = "data/climate_edu/389801eng.pdf"
converter = DocumentConverter()
result = converter.convert(file_path)
doc = result.document
doc_name = doc.name

/opt/homebrew/Caskroom/miniconda/base/envs/ibm/lib/python3.12/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/opt/homebrew/Caskroom/miniconda/base/envs/ibm/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/homebrew/Caskroom/miniconda/base/envs/ibm/lib/python3.12/site-packages/numpy/_core/_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/opt/homebrew/Caskroom/miniconda/base/envs/ibm/lib/python3.12/site-packages/docling/pipeline/standard_pdf_pipeline.py:262: RuntimeWarning: Mean of empty slice
  np.nanmean(


In [ ]:
doc_md = doc.export_to_markdown(page_break_placeholder=PAGE_BREAK, image_placeholder="",)
with open("data/climate_edu/389801eng.md", "w") as f:
    f.write(doc_md)

pages = doc_md.split(PAGE_BREAK)
pages = [markdown_cleanup(page) for page in pages]

## Chunk

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    "ibm-granite/granite-embedding-278m-multilingual"
)
max_length = tokenizer.model_max_length
chunk_overlap_length = int(max_length / 10)
chunk_size_length = max_length - chunk_overlap_length
markdown_splitter = MarkdownTextSplitter.from_huggingface_tokenizer(
    tokenizer=tokenizer,
    chunk_size=chunk_size_length,
    chunk_overlap=chunk_overlap_length
)

In [27]:
chunked_docs = []
for i, page in enumerate(pages):
    chunks = markdown_splitter.split_text(page)
    for j, chunk in enumerate(chunks):
        document = Document(
            page_content=chunk,
            embeddings=tokenizer.encode(chunk, add_special_tokens=False),
            metadata={
                "source": doc_name,
                "page": i + 1,
                "chunk": j + 1
            }
        )
        chunked_docs.append(document)

Token indices sequence length is longer than the specified maximum sequence length for this model (535 > 512). Running this sequence through the model will result in indexing errors


## Ingest

In [28]:
embedder = WatsonxEmbeddings(
    url=url,
    apikey=api_key,
    project_id=project_id,
    model_id="ibm/granite-embedding-278m-multilingual"
)

In [29]:
vector_store = Chroma(
    collection_name="climate_edu",
    persist_directory="data/chroma_db",
    embedding_function=embedder
)

In [30]:
vector_store.add_documents(chunked_docs)

['052cd4dc-fa55-4383-a44c-df3e2756754b',
 'f1fbab8c-9046-4648-bb3d-b8be15a6ca98',
 'e0a00b7b-cfd8-4493-b70c-232fe4f46afb',
 'f5bf98df-7297-446d-924e-ab09649f0818',
 '9f95410e-98d6-4d85-9949-a0698094d611',
 '1b6182c2-8a7e-45a3-bae7-0f2d8bfd0ba4',
 'b076f5fd-a9d9-424e-8105-f2391756e36a',
 '08cfc9e7-249a-40b2-af5c-768b563bcc25',
 'fe6fc973-d90a-4307-8bd4-9435f963a515',
 '63ea5eb5-c1d6-4bbb-a1af-f89f138f4ecd',
 '9727803f-508d-476e-9717-3539e378fd65',
 '89120ba5-810b-45fd-b4ab-da31874c94ba',
 '5ee01814-92c8-4691-959d-fffc72c7f1d3',
 '978c1b1a-bc69-480b-b29d-25e0715124b5',
 'b210104b-9875-4f9c-ac0f-06d40709e356',
 'feb06858-a7ca-4dc5-bf82-9717f3692809',
 '97790c36-ebe3-4bdb-bcde-2f9d06cd29fa',
 'c2681a89-0e71-401a-bd2f-a03a1d087f5a',
 '653807c6-09e0-407c-badf-cb1d7beb1e58',
 '5bc25cfb-b48a-4493-9e4c-05f2d5e47603',
 'c8184681-3541-4bc4-a8c8-7c32319ec7dc',
 '1927c738-4375-425e-97e3-c88ccc7b2059',
 'cdf34aac-fc2a-4ee1-8e78-50bf975f2963',
 'f1131b6c-eceb-496c-93cf-b62af9b4c1ce',
 'fd4f6fe1-04a2-

## Retrieval

In [31]:
user_query = "What could be done by a single person to help mitigate climate change?"

In [32]:
retriever = vector_store.similarity_search(
    user_query,
    k=2,
)

In [ ]:
retriever_docs = [doc.page_content for doc in retriever]
print("Retrieved Documents:")
answer = ""
for i, doc in enumerate(retriever_docs):
    answer += f"Document {i + 1}:\n{doc}\n"

Retrieved Documents:
Document 1:
## Education or knowledge is only one aspect of motivating behaviour change
In 2022, the Intergovernmental Panel for Climate Change concluded that behavioural interventions could increase the efficacy of subsidies and taxes. In their recommendations on decarbonization, the Science Academies of G7 countries urged governments to strengthen climate literacy and citizen involvement and to promote behaviour-oriented science to support transformative social innovations and increase support for technologies, policies and routines for carbon-neutral lifestyles (Jenny and Betsch, 2022).
A review of 10 meta-analyses on climate change mitigation field-based interventions found that the 6 intervention categories (education, feedback, commitment, appeals, financial incentives and social comparison) were generally effective in motivating climate action. Interventions based on social comparisons and financial incentives were the most effective, while those based on ed

## Augmentation

In [48]:
prompt_template = f"""<|begin_of_text|><|header_start|>system<|header_end|>
You are a teacher who provide information on climate change, its impacts, and actions individuals can take to mitigate it. Use the provided context to answer each user question as accurately and briefly as possible. If the context does not contain the answer, respond with: "Sorry, I cannot assist you."
Answer the queries with the same user's input language no matter the context's language.
Always answer in markdown format, using bullet points for lists and bold for important facts. If the context is not sufficient to answer the question, politely inform the user that you cannot assist them.
Keep the the answers like you are teaching and add a little interaction to the answer, like asking a question or giving a suggestion to the user.
Example of a good answer:
Here are some actions a single person can take to help mitigate climate change:
    1. Reduce energy consumption by using energy-efficient appliances and turning off lights when not in use.
    2. Use public transportation, carpool, bike, or walk instead of driving alone.
    3. Reduce, reuse, and recycle to minimize waste.
    4. Support renewable energy sources like solar or wind power.
I hope these suggestions help you contribute to a more sustainable future! What actions do you think you can take to help mitigate climate change?
Context is provided below. Use it to answer the user's question. If the context does not contain the answer, respond with: "Sorry, I cannot assist you."
Context:
{answer}<|eot|>
<|header_start|>user<|header_end|>

{user_query}<|eot|>
<|header_start|>assistant<|header_end|>
"""

In [49]:
print(prompt_template)

<|begin_of_text|><|header_start|>system<|header_end|>
You are a teacher who provide information on climate change, its impacts, and actions individuals can take to mitigate it. Use the provided context to answer each user question as accurately and briefly as possible. If the context does not contain the answer, respond with: "Sorry, I cannot assist you."
Answer the queries with the same user's input language no matter the context's language.
Always answer in markdown format, using bullet points for lists and bold for important facts. If the context is not sufficient to answer the question, politely inform the user that you cannot assist them.
Keep the the answers like you are teaching and add a little interaction to the answer, like asking a question or giving a suggestion to the user.
Example of a good answer:
Here are some actions a single person can take to help mitigate climate change:
    1. Reduce energy consumption by using energy-efficient appliances and turning off lights whe

In [ ]:
model_id = "meta-llama/llama-4-maverick-17b-128e-instruct-fp8"
parameters = {"max_new_tokens": 1024, "decoding_method": "greedy"}
watsonx_llm = WatsonxLLM(
    model_id=model_id,
    url=url,
    apikey=api_key,
    project_id=project_id,
    params=parameters
)

In [ ]:

out = watsonx_llm.invoke(prompt_template)

In [72]:
print(out)

## Individual Actions to Mitigate Climate Change
While the context focuses on broader interventions and strategies, we can still infer some actions a single person can take. Here are a few suggestions based on the information provided:
* **Understand the importance of social comparisons**: Knowing how your actions compare to others can motivate you to change. For example, seeing that your neighbors are reducing their energy consumption might encourage you to do the same.
* **Take advantage of financial incentives**: Many governments and companies offer incentives for behaviors like using renewable energy or reducing waste. Look for these opportunities in your area.
* **Get involved in your community**: Strengthening climate literacy and citizen involvement can support broader social innovations. Consider participating in local climate initiatives or discussions.
* **Make informed choices**: Educate yourself on climate change, but also be aware that knowledge alone may not lead to actio

In [71]:
# Save the output to a markdown file
with open("data/climate_edu/output.md", "w") as f:
    f.write(out)